In [5]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


## 1. Fetching data:

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("varpit94/apple-stock-data-updated-till-22jun2021")

print("Path to dataset files:", path)

/opt/anaconda3/envs/dataenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 205k/205k [00:00<00:00, 7.76MB/s]

Extracting files...
Path to dataset files: /Users/minhnguyen/.cache/kagglehub/datasets/varpit94/apple-stock-data-updated-till-22jun2021/versions/8


In [6]:
!mv "/Users/minhnguyen/.cache/kagglehub/datasets/varpit94/apple-stock-data-updated-till-22jun2021/versions/8" "C:\Users\hweinguyen\Final Project\Architecture\AAPL.csv"

In [14]:
import pandas as pd

df = pd.read_csv('AAPL.csv')

df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100323,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095089,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088110,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090291,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092908,73449600


## 2. Clean data:

In [15]:
df['Date'] = pd.to_datetime(df['Date'])
filter_year_df = df[(df['Date'].dt.year >= 2012 ) & (df['Date'].dt.year <= 2022)]
filter_year_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
7835,2012-01-03,14.621429,14.732143,14.607143,14.686786,12.575917,302220800
7836,2012-01-04,14.642857,14.810000,14.617143,14.765714,12.643503,260022000
7837,2012-01-05,14.819643,14.948214,14.738214,14.929643,12.783874,271269600
7838,2012-01-06,14.991786,15.098214,14.972143,15.085714,12.917513,318292800
7839,2012-01-09,15.196429,15.276786,15.048214,15.061786,12.897022,394024400


In [21]:
filter_year_df.isnull().any()

Date         False
Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

In [22]:
filter_year_df[filter_year_df.duplicated()]

,Date,Open,High,Low,Close,Adj Close,Volume


In [23]:
cleaned_df = filter_year_df.sort_values(by='Date', ascending=True).reset_index(drop=True)
cleaned_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-03,14.621429,14.732143,14.607143,14.686786,12.575917,302220800
1,2012-01-04,14.642857,14.810000,14.617143,14.765714,12.643503,260022000
2,2012-01-05,14.819643,14.948214,14.738214,14.929643,12.783874,271269600
3,2012-01-06,14.991786,15.098214,14.972143,15.085714,12.917513,318292800
4,2012-01-09,15.196429,15.276786,15.048214,15.061786,12.897022,394024400


In [37]:
cleaned_df.to_csv('Cleaned_Apple_Data.csv')

## 3. Put the raw data and cleaned data into HDFS for further use:

In [26]:
%pip install hdfs

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hdfs: filename=hdfs-2.7.3-py3-none-any.whl size=34325 sha256=d2de63ad7a80f9233288b68d1f02e23e321e12a79dedc54d4439268a2513db76
  Stored in directory: /Users/minhnguyen/Library/Caches/pip/wheels/97/ae/d9/536505928dd3a458b206013b02625df8f12d22fa154f2bfd65
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=543d6cd882bec049309e589a9eda21ead7b056595084c0f5aa7b3637a87d0e14
  Stored in directory: /Users/minhnguyen/Library/Caches/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built hdfs docopt
Note: you may need to restart the kernel to use updated packages.


In [28]:
!hdfs version

Hadoop 3.4.0
Source code repository git@github.com:apache/hadoop.git -r bd8b77f398f626bb7791783192ee7a5dfaeec760
Compiled by root on 2024-03-04T06:35Z
Compiled on platform linux-x86_64
Compiled with protoc 3.21.12
From source with checksum f7fe694a3613358b38812ae9c31114e
This command was run using /Users/minhnguyen/hadoop-3.4.0/share/hadoop/common/hadoop-common-3.4.0.jar


In [29]:
!jps

69219 SecondaryNameNode
69429 ResourceManager
68951 NameNode
69641 NodeManager
79711 Jps
69070 DataNode


### a. Raw data to HDFS:

In [36]:
!hdfs dfs -ls /user/minhnguyen/data 

2024-12-07 17:29:28,136 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ls: `/user/minhnguyen/data': No such file or directory


In [44]:
from hdfs import InsecureClient

# Connect to the HDFS 
client = InsecureClient('http://localhost:9870', user='minhnguyen')

# Define the local raw_file cleaned_file path 
raw_data_path = '/Users/minhnguyen/Workspace/FINALPROJECT/ARCHITECTURE/AAPL.csv'
cleaned_data_path = '/Users/minhnguyen/Workspace/FINALPROJECT/ARCHITECTURE/Cleaned_Apple_Data.csv'

# Define the HDFS raw_file, cleaned_file path 
raw_hdfs_path = '/apple_stock_data/raw_data/Apple_Stock_1980_2022.csv'
cleaned_hdfs_path = '/apple_stock_data/cleaned_data/Cleaned_Apple_Stock.csv'

# Upload the file to HDFS
client.upload(raw_hdfs_path, raw_data_path)
client.upload(cleaned_hdfs_path, cleaned_data_path)

# List files in the HDFS directory
files_in_hdfs = client.list('/apple_stock_data')
print("Files in HDFS directory:", files_in_hdfs)

Files in HDFS directory: ['cleaned_data', 'raw_data']
